In [10]:
def train(in_alpha,in_l1_ratio):
    import os 
    import sys 
    import warnings
    
    import pandas as pd 
    import numpy as np 
    from sklearn.metrics import mean_absolute_error, mean_squared_error,\
        r2_score
    from sklearn.model_selection import train_test_split 
    from sklearn.linear_model import ElasticNet 
    
    import mlflow 
    import mlflow.sklearn

    import logging 

    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2

    warnings.filterwarnings("ignore")
    np.random.seed(42)

    csv_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'

    try:
        data = pd.read_csv(csv_url,sep=";")
    except Exception as e:
        logger.exception(f"Unable to fecth the data. Error{e}")

    #Split the data into training and test sets (0.75 and 0.25 split)
    train,test = train_test_split(data,test_size=0.25,random_state = 42)

    # creating the test and train datasets 

    train_x = train.drop(["quality"],axis = 1)
    test_x = test.drop(["quality"],axis= 1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    #set default values if no alpha is provided 
    if float(in_alpha) is None:
        alpha =0.5
    else:
        alpha = float(in_alpha)

    # set default values if no ll _ration is provided 
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    #Useful for multiple runs 
    with mlflow.start_run():
        #Execute ElasticNet
        lr = ElasticNet(alpha = alpha , l1_ratio = l1_ratio ,random_state=42)
        lr.fit(train_x,train_y)

        #Evaluate metrics
        predicted_qualities = lr.predict(test_x)
        (rmse,mae,r2) = eval_metrics(test_y,predicted_qualities)

        #Print out metrics 
        print(f"Elasticnet model: aplha = {alpha} l1_ratio = {l1_ratio}")
        print(f"\nRMSE {rmse}")
        print(f" MAE: {mae}")
        print(f"R2: {r2}")

        # log parameter, metrics and model to MLflow
        mlflow.log_param("alpha",alpha)
        mlflow.log_param("l1_ratio",l1_ratio)
        mlflow.log_param("rmse",rmse)
        mlflow.log_param("r2",r2)
        mlflow.log_param("mae",mae)

        mlflow.sklearn.log_model(lr,"model")

In [11]:
train(0.5,0.5)

Elasticnet model: aplha = 0.5 l1_ratio = 0.5

RMSE 0.7436470916334205
 MAE: 0.6042761768399746
R2: 0.10601910075094545


In [12]:
train(0.2,0.2)

Elasticnet model: aplha = 0.2 l1_ratio = 0.2

RMSE 0.6926302996485334
 MAE: 0.5529215843503876
R2: 0.22447206750598891
